戸上真人「Pythonで学ぶ音源分離」https://amzn.to/3pRqvII をJuliaでやっていく

# 第3章「音源分離で用いられる数学的知識の基礎」

In [1]:
versioninfo()

Julia Version 1.5.3
Commit 788b2c77c1 (2020-11-09 13:37 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i7-7920HQ CPU @ 3.10GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


## 第1節 音源分離で用いる線形代数

### 行列の基本的な演算（p.67）

行列（Matrix）は配列（Array）の一種なのでドキュメントは配列のところか、線形代数の標準パッケージを見るといいかも
- https://docs.julialang.org/en/v1/base/arrays
- https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/

他の言語とJuliaとの違いについては公式サイトの次のリンクがうれしい
- https://docs.julialang.org/en/v1/manual/noteworthy-differences/

In [2]:
# 行列を作る
A = [3 1 2
     2 3 1]

2×3 Array{Int64,2}:
 3  1  2
 2  3  1

In [3]:
# 行列の大きさ・行列の表示
println("size of the matrix: $(size(A))")
println("A = $A")

size of the matrix: (2, 3)
A = [3 1 2; 2 3 1]


In [4]:
# スカラー積
c = 2
println("cA = $(c * A)")

cA = [6 2 4; 4 6 2]


In [5]:
# 和
B = [-1 2 4
      1 8 6]
println("A+B = $(A+B)")

A+B = [2 3 6; 3 11 7]


In [6]:
# 積
B = [ 4 2
     -1 3
      1 5]
println("AB = $(A*B)")

AB = [13 19; 6 18]


In [7]:
# アダマール積（要素ごとの積）
B = [-1 2 4
      1 8 6]
println("A⊙B = $(A.*B)")

A⊙B = [-3 2 8; 2 24 6]


In [8]:
# 行列の転置
println("A^T = $(A')")   # 複素数だとエルミート転置（随伴行列）になるので注意
println("A^T = $(transpose(A))")
println("A^T = $(permutedims(A, [2 1]))")   # swapaxesの代わり

A^T = [3 2; 1 3; 2 1]
A^T = [3 2; 1 3; 2 1]
A^T = [3 2; 1 3; 2 1]


In [9]:
# エルミート転置（随伴行列）
A = [3+0im  1+2im  2+3im
     2+0im  3-4im  1+3im]
println("A^H = $(A')")
println("A^H = $(adjoint(A))")

A^H = Complex{Int64}[3 + 0im 2 + 0im; 1 - 2im 3 + 4im; 2 - 3im 1 - 3im]
A^H = Complex{Int64}[3 + 0im 2 + 0im; 1 - 2im 3 + 4im; 2 - 3im 1 - 3im]


In [10]:
# 行列の積に対するエルミート転置
A = [3+0im  1+2im  2+3im
     2+0im  3-4im  1+3im]
B = [4+4im  2+3im
    -1+1im  3-2im
     1+3im  5+5im]
println("(AB)^H = $((A*B)')")
println("B^H A^H = $(B' * A')")

(AB)^H = Complex{Int64}[2 - 20im 1 - 21im; 8 - 38im -5 - 8im]
B^H A^H = Complex{Int64}[2 - 20im 1 - 21im; 8 - 38im -5 - 8im]


In [11]:
# 単位行列
using LinearAlgebra
println("I = $(I(3))")
X = Matrix{Float64}(I, 3, 3)   # 型指定をするときにはこういう方法も
println("I = $(X)")

I = Bool[1 0 0; 0 1 0; 0 0 1]
I = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]


Juliaには`rot180`、`rotl90`、`rotr90`といった行列を回転させる関数も用意されているので、画像処理をするときに便利かもしれない。

アインシュタインの縮約記法（numpyの`einsum`）はEinsum.jlというパッケージを入れれば実現できる。
- https://github.com/ahwillia/Einsum.jl

まずは

```julia
pkg> add Einsum
```

でパッケージをインストールしておく。実行には`@einsum`マクロを使う。結果の行列を新しく作るときには`:=`で、もともとある行列への上書きであれば`=`にする。今回は前者。

In [12]:
using Einsum
A = [3 1 2
     2 3 1]
B = [ 4 2
     -1 3
      1 5]
@einsum C[m, n] := A[m, k] * B[k, n]

println("AB = $(@einsum C[m, n] := A[m, k] * B[k, n])")

AB = [13 19; 6 18]


numpyの`einsum`よりも数式に近くて直感的な気がする。

マクロなので、何が起きているのかを見たければ`@macroexpand`で覗くことができる。（実行結果は省略）

```julia
@macroexpand @einsum C[m, n] := A[m, k] * B[k, n]
```

### アインシュタインの縮約記法を用いたテンソル計算（p.71）

In [13]:
using Random
Random.seed!(0)

L = 10
K = 5
M = 3
R = 3
N = 3

# 行列の準備
A = rand(L, K, M, R) .+ rand(L, K, M, R) * 1im
B = rand(K, R, N) .+ rand(K, R, N) * 1im

5×3×3 Array{Complex{Float64},3}:
[:, :, 1] =
  0.147976+0.954754im  0.00727965+0.666255im  0.197326+0.387483im
  0.176277+0.454864im     0.23718+0.506862im  0.525695+0.964492im
 0.0450598+0.671545im   0.0261147+0.583576im  0.150151+0.498499im
  0.107693+0.105693im    0.113192+0.252215im  0.842739+0.582479im
  0.766978+0.530588im    0.452783+0.218204im  0.289268+0.127691im

[:, :, 2] =
 0.777691+0.133133im   0.318873+0.976239im  0.342481+0.223719im
 0.999202+0.770051im   0.151057+0.096452im  0.173517+0.962207im
 0.732105+0.0611731im  0.349859+0.115407im  0.935676+0.585847im
 0.866026+0.278388im    0.78054+0.26654im   0.416783+0.613683im
  0.89127+0.218322im   0.935379+0.679839im  0.695759+0.908961im

[:, :, 3] =
  0.960876+0.719184im  0.659973+0.207739im   0.255037+0.222186im
 0.0269988+0.201307im  0.784438+0.873917im   0.789986+0.110189im
  0.291065+0.636779im    0.6848+0.350624im  0.0250795+0.0431595im
  0.377485+0.881304im  0.170158+0.197977im    0.23078+0.929219im
  0.682638+0.11907

In [14]:
# アインシュタインの縮約記法での行列積
@einsum C[l,k,m,n] := A[l,k,m,r] * B[k,r,n]
println("size(C): $(size(C))")

# l=1、k=1について検算
println("A(1,1)B(1,1) = $(A[1,1,:,:] * B[1,:,:])")
println("C(1,1) = $(C[1,1,:,:])")

size(C): (10, 5, 3, 3)
A(1,1)B(1,1) = Complex{Float64}[0.08519151565727716 + 1.028382544479421im 0.6841821945018695 + 0.4063384181585944im 0.9468965370847114 + 0.7712941245620732im; -0.4274032034508901 + 1.1323536450340144im 0.13347465647911683 + 1.4375618614587888im 0.6420341784093513 + 0.8057766208887016im; -0.5046213323412478 + 1.3753361976565792im 0.6959288784853086 + 1.1851455333128402im 0.5451961865159336 + 1.545106781292947im]
C(1,1) = Complex{Float64}[0.08519151565727716 + 1.028382544479421im 0.6841821945018695 + 0.4063384181585944im 0.9468965370847114 + 0.7712941245620732im; -0.4274032034508901 + 1.1323536450340144im 0.13347465647911683 + 1.4375618614587888im 0.6420341784093513 + 0.8057766208887016im; -0.5046213323412478 + 1.3753361976565792im 0.6959288784853086 + 1.1851455333128402im 0.5451961865159336 + 1.545106781292947im]


In [15]:
# 行列積をl、kごとに計算したあと、1方向に和をとる
@einsum C[k,m,n] := A[l,k,m,r] * B[k,r,n]
println("size(C): $(size(C))")

# k=1について検算
C_2 = A[1,1,:,:] * B[1,:,:]
for l in 2:L
    C_2 += A[l,1,:,:] * B[1,:,:]
end
println("C_2(1) = $(C_2)")
println("C(1) = $(C[1,:,:])")

size(C): (5, 3, 3)
C_2(1) = Complex{Float64}[-5.862491672146529 + 9.647132037906514im 0.8803223314751973 + 11.10141687938723im 3.878284908737599 + 11.77750756197981im; -8.273438421296428 + 13.569978907707453im 1.950558482631744 + 15.266004756830288im 5.215925456974465 + 16.08338092415706im; -7.242953190598838 + 11.720011898592752im 1.694357543835014 + 13.538314276703987im 3.46493682809629 + 14.660736722891214im]
C(1) = Complex{Float64}[-5.86249167214653 + 9.647132037906514im 0.8803223314751971 + 11.101416879387228im 3.878284908737599 + 11.77750756197981im; -8.27343842129643 + 13.569978907707455im 1.950558482631744 + 15.26600475683029im 5.215925456974465 + 16.08338092415706im; -7.242953190598836 + 11.720011898592752im 1.694357543835014 + 13.538314276703987im 3.4649368280962896 + 14.66073672289121im]


In [16]:
# 縮約記法でのアダマール積
@einsum C[l,k,m,n] := A[l,k,m,n] * B[k,m,n]
println("size(C): $(size(C))")

# 検算
println("A(1,1).*B(1,1) = $(A[1,1,:,:] .* B[1,:,:])")
println("C(1,1) = $(C[1,1,:,:])")

size(C): (10, 5, 3, 3)
A(1,1).*B(1,1) = Complex{Float64}[0.12016706455109111 + 0.7866463453678985im 0.13301098230011382 + 0.09798481734242734im 0.20686922048424608 + 0.25623053081928887im; -0.008516190506050085 + 0.1309660707563912im 0.08834848131235126 + 1.0201974871449235im 0.0015297093434845088 + 0.6664442481778317im; -0.1042379648936248 + 0.4017825881649155im 0.056638242043394675 + 0.09475188703988718im 0.17820374246790424 + 0.2946268520431004im]
C(1,1) = Complex{Float64}[0.12016706455109111 + 0.7866463453678985im 0.13301098230011382 + 0.09798481734242734im 0.20686922048424608 + 0.25623053081928887im; -0.008516190506050085 + 0.1309660707563912im 0.08834848131235126 + 1.0201974871449235im 0.0015297093434845088 + 0.6664442481778317im; -0.1042379648936248 + 0.4017825881649155im 0.056638242043394675 + 0.09475188703988718im 0.17820374246790424 + 0.2946268520431004im]


## 第2節 逆行列

## 第3節 ベクトル・行列の微分

## 第4節 確率・統計の基礎知識